In [ ]:
%pip install sentence-transformers -q

  Using cached numpy-2.3.1-cp311-cp311-win_amd64.whl.metadata (60 kB)
Using cached numpy-2.3.1-cp311-cp311-win_amd64.whl (13.0 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.4
    Uninstalling numpy-1.23.4:
      Successfully uninstalled numpy-1.23.4
Note: you may need to restart the kernel to use updated packages.


  You can safely remove it manually.

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: C:\Users\anubh\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [1]:
from langchain.chains import LLMChain, RetrievalQA
from langchain.prompts import PromptTemplate
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
import chromadb
from sentence_transformers import SentenceTransformer
from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.llms import CTransformers

C:\Users\anubh\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from langchain_community.llms import Ollama

llm = Ollama(model="llama2")  # or whatever name you used in `ollama run`

C:\Users\anubh\AppData\Local\Temp\ipykernel_5992\425466352.py:3: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="llama2")  # or whatever name you used in `ollama run`


In [3]:
response = llm.invoke("What is the capital of France?")
print(response)



The capital of France is Paris.


In [4]:
#extract the text from a PDF file
def load_pdf(data):
    loader = DirectoryLoader(
        data,
        glob="**/*.pdf",
        loader_cls=PyPDFLoader,
    )
    documents = loader.load()
    return documents

In [5]:
extracted_data = load_pdf(data="data")

In [6]:
#text Chunking
def text_chunking(documents):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=20,
    )
    text_chunks = text_splitter.split_documents(documents)
    return text_chunks

In [7]:
text_chunks = text_chunking(documents=extracted_data)
len(text_chunks)

5859

In [8]:
#download the embedding model from huggingface

def download_huggingfaceEmbeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings


In [9]:
embeddings = download_huggingfaceEmbeddings()

C:\Users\anubh\AppData\Local\Temp\ipykernel_5992\3316817663.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


In [11]:
query_res = embeddings.embed_query("What is the capital of France?")
print(query_res)

[0.08204811811447144, 0.03605549782514572, -0.0038928815629333258, -0.004881042055785656, 0.025651127099990845, -0.057143475860357285, 0.012191563844680786, 0.00467892037704587, 0.0349498875439167, -0.02242197096347809, -0.00800524465739727, -0.10935357213020325, 0.022724736481904984, -0.029320916160941124, -0.04352208599448204, -0.1202412024140358, -0.0008486119331791997, -0.018150130286812782, 0.056129518896341324, 0.0030852793715894222, 0.002336376579478383, -0.016839278861880302, 0.06362469494342804, -0.023660195991396904, 0.031493544578552246, -0.034797921776771545, -0.020548837259411812, -0.002790994942188263, -0.01103801280260086, -0.036126717925071716, 0.05414106324315071, -0.03661714866757393, -0.025008678436279297, -0.038170378655195236, -0.0496036522090435, -0.015148064121603966, 0.02131502889096737, -0.012740440666675568, 0.07670093327760696, 0.04435573145747185, -0.010834850370883942, -0.029760019853711128, -0.01697046123445034, -0.024691861122846603, 0.008087079972028732,

In [12]:
# passing text to chromadb
from langchain_community.vectorstores import Chroma


vectorstore = Chroma.from_documents(
    documents=text_chunks,
    embedding=embeddings,
    persist_directory="./chroma_db"
)

vectorstore.persist()

C:\Users\anubh\AppData\Local\Temp\ipykernel_5992\3433834329.py:11: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectorstore.persist()


In [19]:
#setting up retriever
retriever = vectorstore.as_retriever(search_kwargs={"k": 1})

def query_vectorstore(query):
    response = retriever.invoke(query)
    return response

query = "What are the symptoms of diabetes?"
response = query_vectorstore(query)
response

[Document(metadata={'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'moddate': '2004-12-18T16:15:31-06:00', 'total_pages': 637, 'creator': 'PyPDF', 'page_label': '438', 'source': 'data\\Medical_book.pdf', 'page': 437, 'creationdate': '2004-12-18T17:00:02-05:00'}, page_content='• Type I diabetes mellitus. Characterized by fatigue and\nan abnormally high level of glucose in the blood\n(hyperglycemia).\n• Amyotrophic lateral schlerosis. First signs are stum-\nbling and difficulty climbing stairs. Later, muscle\ncramps and twitching may be observed as well as\nweakness in the hands making fastening buttons or\nturning a key difficult. Speech may become slowed or\nslurred. There may also be difficluty swallowing. As\nrespiratory muscles atrophy, there is increased danger')]

In [20]:
TEMPLATE="""
Act as a medical and pharmaceutical expert. You are given a question and a set of documents as context.
Your task is to answer the question based on the context provided. If the context does not provide enough information, you should state that you do not have enough information to answer the question.
Question: {question}
Context: {context}
Answer:
"""

prompt = PromptTemplate(
    input_variables=["question", "context"],
    template=TEMPLATE)
chain_type_kwargs={"prompt" : prompt}


In [25]:
from langchain_community.llms import Ollama

llm = Ollama(
    model="llama2",
    temperature=0.7,
    top_p=0.9,
    repeat_penalty=1.1,
    num_predict=512  # Use this instead of max_tokens
)

In [26]:
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs=chain_type_kwargs)

In [28]:
while True:
    query = input("Enter your question (or 'exit' to quit): ")
    if query.lower() == 'exit':
        break
    response = qa.invoke(query)
    print("Response:", response['result'])
    print("Source Documents:", response['source_documents'])  # Display source documents if needed

Response: Based on the context provided, there are several diseases that can be caused by obesity. These include:

1. Type 2 diabetes: Obesity is a major risk factor for developing type 2 diabetes, as excess body fat can increase resistance to insulin and lead to high blood sugar levels. (Source: Berg, F. M. "Eating Disorders Affect Both the Mind and Body." Healthy Weight Journal 9, no. 2 (1995): 27-31.)
2. Cardiovascular disease: Obesity can increase the risk of heart disease by raising blood pressure, cholesterol levels, and inflammation, which can lead to heart attacks, strokes, and other cardiovascular problems. (Source: Kubersky, R. Everything You Need to Know about Eating Disorders. New York: The Rosen Publishing Group, Inc., 1992.)
3. Sleep apnea: Obesity can cause sleep apnea, a condition in which a person's breathing is interrupted during sleep due to the collapse of the soft tissues in the throat. (Source: Kubersky, R. Everything You Need to Know about Eating Disorders. New Y